In [3]:
import json
import numpy as np
import keras
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
import json
import keras
import pandas as pd
import keras.preprocessing.text as kpt
from keras.preprocessing.text import Tokenizer
import numpy as np
from sklearn.model_selection import train_test_split

from keras.models import Sequential
from keras.layers import Dense, LSTM, SimpleRNN, Dropout, Activation, Embedding
from datetime import datetime
from tensorflow.keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

import json
from keras.models import model_from_json

# Import libraries
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import nltk 
# nltk.download()
import string
import re
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

import string
import re
from keras.preprocessing.text import Tokenizer

In [4]:
# we're still going to use a Tokenizer here, but we don't need to fit it
tokenizer = Tokenizer(num_words=10000)
# for human-friendly printing
labels = ['fake', 'real']

# read in our saved dictionary
with open('./Dictionary_Models/glove_models_dictionary.json', 'r') as dictionary_file:
    dictionary = json.load(dictionary_file)

In [5]:
def convert_text_to_index_array(text):
    words = kpt.text_to_word_sequence(text)
#     words = pad_sequences(words, padding='post', maxlen=23)
#     print(text)
    wordIndices = []
    for word in words:
        if word in dictionary:
            wordIndices.append(dictionary[word])
    return wordIndices

In [8]:
json_file = open('./NN_Models/Glove_BLSTM_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()

model = model_from_json(loaded_model_json)

model.load_weights('./NN_Models/Glove_BLSTM_model.h5')


In [9]:
tweetText = 'there is a fire at melville mountain'

tweetText = convert_text_to_index_array(tweetText)

input = pad_sequences([tweetText], padding='post', maxlen=100)
    
pred = model.predict_generator(input)

def rounding(results):
    '''Results needs to be rounded to 0 or 1 for fake or real, respectively'''
    if results < 0.5:
        return 0
    else:
        return 1
    
predictions_final = [rounding(x) for x in pred]

predictions_final

Instructions for updating:
Please use Model.predict, which supports generators.


[1]

# Connecting to Tweepy

In [10]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import API
from tweepy import Cursor

import sys  
sys.path.insert(0, './Python_Models/')
import credentials

In [11]:
class TwitterAuthenticator():
    def authenticate_twitter_app(self):
        auth = OAuthHandler(credentials.CONSUMER_KEY, credentials.CONSUMER_SECRET)
        auth.set_access_token(credentials.ACCESS_TOKEN, credentials.ACCESS_TOKEN_SECRET) 
        return auth       

In [12]:
class TweeterStreamer():
    def __init__(self):
        self.twitter_authenticator = TwitterAuthenticator()
      
    def stream_tweets(self, fetched_tweet_file, hash_tag_list):
        #handles twitter authentication and twitter streaming API
        listener = TwitterListener(fetched_tweet_file) 
        auth = self.twitter_authenticator.authenticate_twitter_app()       
        stream = Stream(auth, listener)
        
        stream.filter(track=hash_tag_list)
         

In [13]:
def rounding(results):
    '''Results needs to be rounded to 0 or 1 for fake or real, respectively'''
    if results < 0.5:
        return 0
    else:
        return 1

In [14]:
class TwitterListener(StreamListener):  
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename
    
    def on_data(self, data):
        try:
            tweetText = json.loads(data)['text']
            testArr = convert_text_to_index_array(tweetText)
            input = pad_sequences([testArr], padding='post', maxlen=100)
            
            pred = model.predict_generator(input)

            predictions_final = [rounding(x) for x in pred]
            
            print('\n',tweetText)
            
            print('****',labels[predictions_final[0]], 'Tweet')
            
            # print("%s \n the %s sentiment; %f%% confidence" % (tweetText ,labels[np.argmax(pred)], pred[0][np.argmax(pred)] * 100))
      
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on data: %s" % str(e))
        return True
    
    def on_error(self, status):
        if status == 420:
            return False
        print(status)
        
        

In [18]:
if __name__ == "__main__":
    mylist =['hazard','fire','ablaze','accident','aftershock','ambulance']   
    fetched_tweets_filename = "text.txt"
    
    tweeter_streamer = TweeterStreamer()
    tweeter_streamer.stream_tweets(fetched_tweets_filename,mylist)
    


 RT @WhbJzgkyolRYKPO: #拡散希望
#保護猫
クラウドファンディング残り28日です。
目標金額達成まで、あと30%必要です。
保護された猫ちゃん達に、お力を分けていただけないでしょうか🙇
1000円から御支援できます。
御支援・拡散お願い致します🙇 https…
**** fake Tweet

 RT @lulu_fip: 経過観察9日目

朝からあえてドライだけにしたのでよく噛んで食べてます🍚
凄い勢いで(笑)

#クラウドファンディング
いよいよ残り1日🔥
達成率83%🔥

あと17%です！

あと少しだけ
拡散やご支援のご協力お願いします🙇‍♀️⤵︎⤵︎⤵︎
ht…
**** fake Tweet

 RT @KASHINNISMF: Wassup twitter. Ion tweet much but i recently got in a life threatening accident. I broke my spine, lost my new car, lost…
**** fake Tweet

 RT @khchenlebar: psa hey nctzens contrary to your beliefs, chenle CAN actually cook🤍!! hes not that dumb kid u think that will cause fire o…
**** fake Tweet

 RT @dianagallegos__: Car accident ptsd is probably the worst
**** real Tweet

 @theonlyrymac This aint nothing..... https://t.co/IZVtz07fz9 Try this one....
**** fake Tweet

 RT @FawnMacMT: meet Margaret age 25 ANTIFA MEMBER set 5 police cars on fire she is charged with 5 FEDERAL COUNTS OF ARSON!!
🤣🤣🤣🤣🤣 https://t…
**** real Tweet

 RT @NoToDeforesting: The most


 RT @itsslia_: el cocoppa que daño ha hecho
**** fake Tweet

 RT @MariaBF1: #NowPlaying Carmilla @CarmillaBand - Blood Of Fire on @RockOnTheRise radio with your host @Shamas_Maximus 
https://t.co/gHfHe…
**** fake Tweet

 @sarifidris Forged in fire nih
**** fake Tweet

 jisung’s drink goes from empty to full (from what i can tell), covered in the same green lighting/fire we see in th… https://t.co/KLNeUfq8py
**** fake Tweet

 RT @MazharAbbasGEO: Verdict in high profile Baldia factory fire case in which 264 workers burnt alive at 10.30 to 11 am, today.
**** real Tweet

 RT @WrecklessGamer: Have you written to your MP yet?  I have penned a letter by email today and sent it off.  If you need a template that y…
**** fake Tweet

 RT @Soumik_Pyne: Fun thing is that they used E-VAHAN software which stores the numbers the requests came from. These numbers were linked to…
**** fake Tweet

 RT @muyal123: @TandonRaveena @iujjawaltrivedi #BoycottBollywoodDruggies 
"These druggies are responsible f


 RT @foIklorealbum: taylor please talk to me about portrait of a lady on fire youre not even that famous
**** fake Tweet

 RT @AskAnshul: Now, it's revealed that in Delhi riots, some rioters had first checked the e-Vaahan website before setting vehicles on fire.…
**** real Tweet

 RT @AskAnshul: Now, it's revealed that in Delhi riots, some rioters had first checked the e-Vaahan website before setting vehicles on fire.…
**** real Tweet

 don't call yourself an "animal lover" when you LITERALLY grabbed all your clothes before your pets while your house… https://t.co/jXptoiqNdL
**** fake Tweet

 RT @kkk011102: 【宝石を楽しむサロン】

宝石を少しずつ
集める楽しさ。

毎月のちょっとした楽しみに。

初月無料で
アメジストプレゼント中🎁

https://t.co/I9uEcfTy54 https://t.co/9RntJ8v68P
**** fake Tweet

 RT @Zinny705: Hut people are about to cry premium tears. This tag is about to set the whole tweeter on fire. Anyway we are here to see thei…
**** fake Tweet

 RT @OscarMbo: Let me tell you this for free, this remix is fire!
**** fake Tweet

 RT @Lindse


 RT @PATTI_THUNDER: I absolutely LOVE this picture.  I adore these two.  🥰
**** fake Tweet

 🥺🥺🥺❤💛
**** fake Tweet

 Entrepreneurs on Fire with John Lee Dumas https://t.co/ktE2YJp313
**** fake Tweet

 @LarryGale00 @missgemcollins You're an ambulance chasing diva!
**** fake Tweet

 RT @Peacefu24821167: Are we fire ... yes i think we are ..as this much speed oh my lord I didn't expect.. this is trending with 104K ++ #St…
**** fake Tweet

 This means EVERYTHING.  Thank you sis
**** fake Tweet

 RT @guardian: 'We should be ashamed': Lord Dubs berates UK ministers over camp fire refugees https://t.co/qtQChVE4Xa
**** real Tweet

 RT @velvetjays: And what if we get a Jay and Ni-ki dance break during MAMA and MMA like what happened when they were Parts 1 and 2 in the g…
**** fake Tweet

 RT @AskAnshul: Now, it's revealed that in Delhi riots, some rioters had first checked the e-Vaahan website before setting vehicles on fire.…
**** real Tweet

 В большинстве случаев "популярность" приводит к п


 @HenryMigo @lsdinthemorning @xbroco As people, we only advance if we talk and discuss things. This is just one of t… https://t.co/2rRKQlXOMp
**** fake Tweet

 RT @I30mki: So, we are all just going to ignore the fact that #Iran reverse engineered the #Italian Trums-T fire control system with #Syria…
**** real Tweet

 RT @AskAnshul: Now, it's revealed that in Delhi riots, some rioters had first checked the e-Vaahan website before setting vehicles on fire.…
**** real Tweet

 https://t.co/RcrAi0ggIp #sketch #sketchbook #paper #pen #pencil #artsy #instaart #beautifuleyes
**** fake Tweet

 APRIL Jinsol, posing is iconic. Girl on Fire, indeed 😍🔥 @APRIL_DSPmedia @1stlookofficial https://t.co/rKiHGHJaAE
**** real Tweet

 RT @CaseyDancing: casey frey dancing to september by earth, wind &amp; fire https://t.co/PAodGhH5Pl
**** real Tweet

 RT @cuban_manny: #DemocratsWakeUp this BITCH threatens Mitch McConnell “We Need To Tell Hlm That He Is Playing With Fire 🔥”, #America can y…
**** fake Tweet



...


 RT @AskAnshul: Now, it's revealed that in Delhi riots, some rioters had first checked the e-Vaahan website before setting vehicles on fire.…
**** real Tweet

 #NowPlaying Tedashii - Fire Away Tedashii https://t.co/zaB00rjhHe
**** fake Tweet

 RT @RobertAlai: Sports Ministry Under Fire For Failing To Lift Ban On Football Activities https://t.co/UFeHoMXtEE
**** real Tweet

 @diarioas El problema del gol se resuelve con Asensio y Hazard a tope. Junto a Benzema te aseguras mucho gol. Ademá… https://t.co/juINf17tWh
**** fake Tweet

 RT @AndrewLoveUS: Can she be impeached? They do nothing but tear down our country.

RT if you agree with me!

https://t.co/Np2N93v0KW
**** fake Tweet

 RT @nutsaremixed: If you practice the rhythm method of contraception.... 

.. doing it to fire starter by the prodigy could be the best and…
**** fake Tweet

 The accident happened yesterday evening.
**** real Tweet

 RT @RainyDayRibbons: Shifting silhouettes
a hint of sunset
in the colour of her dress
her ever


 RT @advmonikaarora: Rioters had checked the ownership of vehicles through e-Vaahan portal before setting them on fire https://t.co/s0H3TTVK…
**** real Tweet

 @DanielEliesen Rinse, wash, repeat... I like Flores but in the end the BS spoken needs to equal wins. He is a coach… https://t.co/sHhabRRPrh
**** fake Tweet

 RT @FortuneUSNews: 🇺🇸 WHO'S BLOWING 💩 UP IN CHINA?

🇨🇳China simulates blowing up GUAM
🇨🇳Then CHINA has MULTIPLE EXPLOSIONS IN BEIJING
🇨🇳U.S…
**** real Tweet

 @ragdollplaid But the fearful, and unbelieving, and the abominable, and murderers, and whoremongers, and sorcerers,… https://t.co/VPpxGCAq5N
**** fake Tweet

 Hazard on the road to Staffin from Portree: cows on the road, broken free from a field. Please beware. Caller just… https://t.co/i1G3OXvgHl
**** real Tweet

 If @RockstarGames has some balls they would make the car slightly faster if you put the fire livery on it 😆… https://t.co/sATYbV3CWb
**** fake Tweet

 RT @NCITYofc: ㅤ
ㅤ
TO THE WORLD!

NCITY 2020 CHAPTER O


 RT @DeputyAlvin: Damn hearts out for my buddy who is a police officer in El Paso. He was in a car accident while on duty pursuing a kidnapp…
**** real Tweet

 RT @FawnMacMT: meet Margaret age 25 ANTIFA MEMBER set 5 police cars on fire she is charged with 5 FEDERAL COUNTS OF ARSON!!
🤣🤣🤣🤣🤣 https://t…
**** real Tweet

 @UltimateSmashBR @william_saneto Fire Emblem 3 cabeças é bom demais man
**** fake Tweet

 RT @Princemudi_: Its only in Nigeria you will see FOOD IS READY..but when you go in &amp; ask for pounded yam with egusi, they will tell you is…
**** fake Tweet

 Dear @CholaMS @CholaMSRisk @Cholainsurance 
My car got accident yesterday. I tried to contact your Toll free Number… https://t.co/vTnd1GAtKR
**** real Tweet

 RT @yuichifr: 僧帽弁閉鎖不全症で苦しむ小さなポメチワ犬ロビンのクラウドファンディング、残り３日です。どうかご支援宜しくお願いいたします…！
https://t.co/VWKnr7l2qB
#拡散希望
#拡散RT希望 
#いぬのいる暮らし https://t.co/…
**** fake Tweet

 RT @antoguerrera: Perugia is definitely on fire. 🔥 According to a police investigation, the Italian language 


 #WXIITRAFFIC Accident with injuries reported on Tanglewood Park Road and North Trail Road in Clemmons.
**** real Tweet

 RT @MercedesAMGF1: Got an #ItalianGP question for us? 

Fire us a tweet and we’ll answer as many as possible during #FP1 📲 

As always, all…
**** fake Tweet

 大学生が営むナゴヤのカレー屋🍛 #八Ｏ吉 さんが #クラウドファンディング に挑戦中🔥

／
📣支援者数1️⃣0️⃣0️⃣人突破💥💥💥
＼

プロジェクト終了まであと8日⚡️⚡️
支援はまだまだ受け付けております！
応援・拡散よろ… https://t.co/UJkfmjVrmk
**** fake Tweet

 Amazon Fire HD 10の実機レビュー － Web会議や周辺機器を接続して仕事に使えるかを試してみました https://t.co/dg0EJrkY5e
**** fake Tweet

 RT @Muriel_FUSI: Dans les mois à venir, des cerfs seront poussés à bout comme samedi à Compiègne, des chiens et chats seront criblés de bal…
**** fake Tweet

 RT @KASHINNISMF: Wassup twitter. Ion tweet much but i recently got in a life threatening accident. I broke my spine, lost my new car, lost…
**** fake Tweet

 @FarziCricketer Watson likely to fire sooner rather than later
**** fake Tweet

 RT @FawnMacMT: meet Margaret age 25 ANTIFA MEMBER set 5 polic

KeyboardInterrupt: 